In [1]:
# extract data around the theatrical release of australian film from the screen australia website.

from requests_html import HTMLSession
from requests_html import HTML
from bs4 import BeautifulSoup
import json
import pandas
import pathlib
import pydash
import uuid

def remove_see(row):

    ''' Remove stray prefixes. '''

    title = row['title'].replace('\r\n\t\t\t', ' ')
    if title[:3] != 'See':
        return title.strip()
    else:
        return title[3:].strip()

def extract_coprod(row):

    ''' Extract coproduction info from title. '''

    if '(' in row['title']:
        splinter = row['title'].split('(')
        if 'Australia' in splinter[1]:
            countries = splinter[1].split(')')[0]
            countries = countries.split('/')
            countries = [x for x in countries if x != 'Australia']
            return '-'.join(countries)

    return ''

def remove_coprod(row):

    ''' Remove coproduction info from title. '''

    if row['coprod'] != '':
        return row['title'].split('(')[0].strip()
    else:
        return row['title']

dataframe = pandas.DataFrame(columns=['year', 'country', 'title', 'distributor', 'reference'])

home = "https://www.screenaustralia.gov.au/fact-finders/cinema/australian-films/feature-film-releases/titles-released"
session = HTMLSession()
text = session.get(home).text

data_links = list()
for x in text.split('<h3 class="tabletitle2">')[1:-1]:
    for desc in ['data-title="Screen Australia: Australian feature film titles released in cinemas, ', 
        'data-title="Screen Australia: Australia feature film titles released in cinemas, ']:
        if desc in x:
            year = x.split(desc)[1].split('"')[0]
            if 'data-id="' in x:
                data_id = x.split('data-id="')[1].split('" data-title')[0]
                data_links.append({'link':data_id, 'year':year})

for x in data_links:
    path = f"https://e.infogram.com/{x['link']}?parent_url=https%3A%2F%2Fwww.screenaustralia.gov.au%2Ffact-finders"
    path += "%2Fcinema%2Faustralian-films%2Ffeature-film-releases%2Ftitles-released&amp;src=embed#async_embed"
    session = HTMLSession()
    r = session.get(path)

    section = r.text.split('<script>window.infographicData=')[1].split(';</script>')[0]
    element = json.loads(section)

    with open(pathlib.Path.home() / 'Desktop' / 'test.json', 'w') as output:
        json.dump(element, output)

    countries = pydash.get(element, 'elements.0.sheetnames')
    for n, y in enumerate(pydash.get(element, 'elements.0.data')):
        y = y[1:]
        for z in y:
            dataframe.loc[len(dataframe)] = [(x['year']), (countries[n])]+[a.replace('\r', '') for a in z]+[(home)]

print(len(dataframe))
dataframe.head(10)

643


,year,country,title,distributor,reference
0,2021,Australia,Akoni,Independent,https://www.screenaustralia.gov.au/fact-finder...
1,2021,Australia,Ascendant,Maslow Entertainment,https://www.screenaustralia.gov.au/fact-finder...
2,2021,Australia,Buckley's Chance,Transmission,https://www.screenaustralia.gov.au/fact-finder...
3,2021,Australia,Disclosure,Bonsai Film,https://www.screenaustralia.gov.au/fact-finder...
4,2021,Australia,The Dry,Roadshow,https://www.screenaustralia.gov.au/fact-finder...
5,2021,Australia,A Family,Pivot Pictures,https://www.screenaustralia.gov.au/fact-finder...
6,2021,Australia,Great White,Bonsai Film,https://www.screenaustralia.gov.au/fact-finder...
7,2021,Australia,Greenfield,Halo Films,https://www.screenaustralia.gov.au/fact-finder...
8,2021,Australia,High Ground,Madman Cinema,https://www.screenaustralia.gov.au/fact-finder...
9,2021,Australia,June Again,StudioCanal,https://www.screenaustralia.gov.au/fact-finder...


In [2]:
session = HTMLSession()
r = session.get(home)

for x in r.text.split('<table')[1:]:
    year = x.split('Cinema release titles by country,')[1].split('"')[0].strip()
    body = x.split('tbody')
    for y in body:  
        html = HTML(html=y)
        if html.find('.cellrowgrouphead', first=True):
            country = html.find('.cellrowgrouphead', first=True).text
            for c in y.split('<tr>'):
                if len(c.split('</td>')) == 3:
                    line = [BeautifulSoup(x, "lxml").text for x in c.split('</td>')]
                    line = [x for x in line if x != '\n']
                    if (len(line)) != 2:
                        raise Exception('number does match')
                    dataframe.loc[len(dataframe)] = [(year), (country)]+line+[(home)]

print(len(dataframe))
dataframe.head(10)

1429


,year,country,title,distributor,reference
0,2021,Australia,Akoni,Independent,https://www.screenaustralia.gov.au/fact-finder...
1,2021,Australia,Ascendant,Maslow Entertainment,https://www.screenaustralia.gov.au/fact-finder...
2,2021,Australia,Buckley's Chance,Transmission,https://www.screenaustralia.gov.au/fact-finder...
3,2021,Australia,Disclosure,Bonsai Film,https://www.screenaustralia.gov.au/fact-finder...
4,2021,Australia,The Dry,Roadshow,https://www.screenaustralia.gov.au/fact-finder...
5,2021,Australia,A Family,Pivot Pictures,https://www.screenaustralia.gov.au/fact-finder...
6,2021,Australia,Great White,Bonsai Film,https://www.screenaustralia.gov.au/fact-finder...
7,2021,Australia,Greenfield,Halo Films,https://www.screenaustralia.gov.au/fact-finder...
8,2021,Australia,High Ground,Madman Cinema,https://www.screenaustralia.gov.au/fact-finder...
9,2021,Australia,June Again,StudioCanal,https://www.screenaustralia.gov.au/fact-finder...


In [3]:
dataframe['title'] = dataframe.apply(remove_see, axis=1)
dataframe['coprod'] = dataframe.apply(extract_coprod, axis=1)
dataframe['title'] = dataframe.apply(remove_coprod, axis=1)

dataframe.loc[dataframe.distributor.isin(['Producer', 'Producer/director']), 'distributor'] = 'harry'
dataframe['distributor'] = dataframe['distributor'].str.split('/')
dataframe = dataframe.explode('distributor')
dataframe['distributor'] = dataframe['distributor'].str.strip()

dataframe['coprod'] = dataframe['coprod'].str.split('-')
dataframe = dataframe.explode('coprod')
dataframe['coprod'] = dataframe['coprod'].str.strip()

work_uuid = dataframe.copy()
work_uuid = work_uuid[['title']].drop_duplicates()
work_uuid['work_uuid'] = [str(uuid.uuid4()) for x in range(len(work_uuid))]
dataframe = pandas.merge(dataframe, work_uuid, on='title', how='left')

distro_uuid = dataframe.copy()
distro_uuid = distro_uuid[['distributor']].drop_duplicates()
distro_uuid['distributor_uuid'] = [str(uuid.uuid4()) for x in range(len(distro_uuid))]
dataframe = pandas.merge(dataframe, distro_uuid, on='distributor', how='left')

dataframe['distro_uuid'] = [str(uuid.uuid4()) for x in range(len(dataframe))]

dataframe.to_csv(pathlib.Path.cwd() / 'screen-australia.csv', index=False)
print(len(dataframe))
dataframe.head(10)

1502


,year,country,title,distributor,reference,coprod,work_uuid,distributor_uuid,distro_uuid
0,2021,Australia,Akoni,Independent,https://www.screenaustralia.gov.au/fact-finder...,,c5107176-9d1d-4ddd-bd7f-f1d0d741c4b3,c30f253c-2cbf-4e01-ab54-56d346bd212f,ecaa22bc-2b3c-4d6d-91f4-13f9a98fc65b
1,2021,Australia,Ascendant,Maslow Entertainment,https://www.screenaustralia.gov.au/fact-finder...,,ac8b6698-0d4f-4422-a8b0-6ce3ec11391e,50fe2394-49ff-4f11-b56c-cd9e25424bad,23030c53-7b31-44cd-a207-2f313c3040cc
2,2021,Australia,Buckley's Chance,Transmission,https://www.screenaustralia.gov.au/fact-finder...,,b0288c54-ca06-4cbd-8f1b-61fc2a45a075,57ffb273-9d20-4537-b16a-098a8295cd68,ed2ba22b-388f-485e-9b10-075a303ca6d0
3,2021,Australia,Disclosure,Bonsai Film,https://www.screenaustralia.gov.au/fact-finder...,,e55637d9-623b-47e3-97e2-4474ccc0c200,945e95e1-3887-438a-a3c9-93f3019b7d8c,94a3a906-f922-4aac-9bda-5f157b0bb8e7
4,2021,Australia,The Dry,Roadshow,https://www.screenaustralia.gov.au/fact-finder...,,c701d9aa-731f-4a59-b141-2690543bb2b8,df3a0ecc-57bd-4bd8-b0f4-5d9d9c03d1ea,f556d99d-0eff-4acb-961a-840328818dc9
5,2021,Australia,A Family,Pivot Pictures,https://www.screenaustralia.gov.au/fact-finder...,,0385dccc-8a61-4ee8-a75b-d1d2ccc1d5c4,88e9ca16-6684-468d-b5d6-efc79e94ed94,340a74a9-eef9-4976-961b-109c08d8baa9
6,2021,Australia,Great White,Bonsai Film,https://www.screenaustralia.gov.au/fact-finder...,,16e9492b-0ca7-453a-8572-272ff5e4a4c3,945e95e1-3887-438a-a3c9-93f3019b7d8c,61183885-b3cc-48bc-9c05-1bfbd6d8a0a0
7,2021,Australia,Greenfield,Halo Films,https://www.screenaustralia.gov.au/fact-finder...,,600bc115-cbd8-417d-a129-08ed3b21552d,4868355f-c62f-4eaf-8729-e2216fb1655c,a05bae1a-0b83-4305-8acc-e35e74b1ba34
8,2021,Australia,High Ground,Madman Cinema,https://www.screenaustralia.gov.au/fact-finder...,,97663c3f-e706-475b-892f-9b0795723dfb,338853a0-1c50-4222-afdb-059029463a80,840069ef-ea23-467c-a98c-663db6a294df
9,2021,Australia,June Again,StudioCanal,https://www.screenaustralia.gov.au/fact-finder...,,d6eddc6f-b70e-4246-8031-9c095dc69fdf,ebdea14f-dd6e-4ab7-95c8-2430d314e7e0,3264400f-5858-45f0-be93-2d5b3e74212a
